In [19]:
from sklearn.svm import OneClassSVM
from sklearn.datasets import make_blobs
from numpy import quantile, where, random
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_json('working/audio_features.json')
df_nested_list = pd.json_normalize(df, record_path =['features'])

TypeError: string indices must be integers

<bound method Series.isna of fan/id_00\abnormal\00000000.wav    {'features': [-23.14673425319661, 2.9508236964...
fan/id_00\abnormal\00000001.wav    {'features': [-23.075589556815537, 2.814327686...
fan/id_00\abnormal\00000002.wav    {'features': [-23.73934065025545, 2.9942088625...
fan/id_00\abnormal\00000003.wav    {'features': [-22.748752785911215, 2.787962125...
fan/id_00\abnormal\00000004.wav    {'features': [-22.924125789369917, 2.746891582...
                                                         ...                        
fan/id_00\normal\00001006.wav                                                    NaN
fan/id_00\normal\00001007.wav                                                    NaN
fan/id_00\normal\00001008.wav                                                    NaN
fan/id_00\normal\00001009.wav                                                    NaN
fan/id_00\normal\00001010.wav                                                    NaN
Name: abnormal, Length: 1418, dtype:

In [8]:
df_nested_list.head()

NameError: name 'df_nested_list' is not defined

In [ ]:
def train_test(train_size,device='fan',id=0):

    # function to label + split audio data

    dir,label = audiodir(device,id)
    dir_abnormal,label_abnormal = audiodir(device,id,Data='abnormal')

    dataset_normal = MIMII(dir,label)
    dataset_abnormal = MIMII(dir_abnormal,label_abnormal)
    train_dataset, test_normal_dataset = torch.utils.data.random_split(dataset_normal, [int(len(dataset_normal)*train_size), len(dataset_normal)- int(len(dataset_normal)*train_size)])

    test_dataset = ConcatDataset([test_normal_dataset, dataset_abnormal])

    Train = torch.utils.data.DataLoader(train_dataset, shuffle=True,num_workers=4,drop_last=True)
    Test = torch.utils.data.DataLoader(test_dataset, shuffle=True,num_workers=4)
    return Train,Test

In [1]:
Train,Test = train_test(0.75,device='fan',id=0)

NameError: name 'train_test' is not defined

In [2]:


def MelSpec(y, sr):
    S = torchaudio.transforms.MelSpectrogram(sample_rate=sr, n_fft=2048, hop_length=512, f_max=sr / 2, norm='slaney',
                                             mel_scale='slaney')(y)
    S_dB = torchaudio.transforms.AmplitudeToDB()(S)
    return S_dB


def mfcc_extract(filename):
    try:
        y, sr = librosa.load(filename, sr=44100)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=int(0.02 * sr), hop_length=int(0.01 * sr))
        return mfcc
    except:
        return